# Holt-Winters-Verfahren
### Prognose-1
 
**Interaktiver Plot zur Visualisierung folgender Verfahren anhand eines beispielhaften Datensatzes**


- exponentielle Glättung erster Ordnung
- Holt-Verfahren (exp. Gl. 2ter Ord.)
- Holt-Winters-Verfahen (exp. Gl. 3ter Ord.)


Beispielhafte Salesdaten wurden aus dem Buch "Operative Planung in ITSystemen" von Frank Herrmann genommen.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Checkbox
from matplotlib.widgets import CheckButtons


# Salesdaten aus Buch Herrmann
seasonal_period = 4
data = pd.DataFrame({
    "Sales": np.array([139, 260, 151, 63, 62, 212, 224, 183, 143, 291, 261, 213, 174, 312, 229, 151, 197, 370, 390, 371, 231, 444, 423, 354])
})


# Glättung erster Ordnung
def ses(data, alpha):
    """Einfache exponentielle Glättung (SES)"""
    ses_values = [data[0]]  # Initialwert
    for t in range(1, len(data)):
        ses_values.append(alpha * data[t] + (1 - alpha) * ses_values[-1])
    return np.array(ses_values)


def holt(data, alpha, beta):
    """Holt-Verfahren für trendangepasste Glättung"""
    niveau = np.mean(data[:5])      # Durchschnitt der ersten 5 Werte für bessere initialisierung
    trend = (data[5] - data[0]) / 5     # hier auch
    holt_values = [niveau + trend]
    
    for t in range(1, len(data)):
        previous_niveau = niveau
        niveau = alpha * data[t] + (1 - alpha) * (niveau + trend)
        trend = beta * (niveau - previous_niveau) + (1 - beta) * trend
        holt_values.append(niveau + trend)
        
    return np.array(holt_values)


def holt_winters(data, alpha, beta, gamma, season_length):
    """Holt-Winters-Verfahren für saisonale und trendangepasste Glättung"""
    niveau = np.mean(data[:5])      # Durchschnitt der ersten 5 Werte für bessere initialisierung
    trend = (data[5] - data[0]) / 5     # hier auch
    seasonals = [data[i] - niveau for i in range(season_length)]
    hw_values = [niveau + seasonals[0]]
    
    for t in range(1, len(data)):
        previous_niveau = niveau
        niveau = alpha * (data[t] - seasonals[t % season_length]) + (1 - alpha) * (niveau + trend)
        trend = beta * (niveau - previous_niveau) + (1 - beta) * trend
        seasonals[t % season_length] = gamma * (data[t] - niveau) + (1 - gamma) * seasonals[t % season_length]
        hw_values.append(niveau + trend + seasonals[t % season_length])
        
    return np.array(hw_values)


# Interaktiver Plot
def plot_interactive(alpha=0.3, beta=0.3, gamma=0.1, show_ses=False, show_holt=False, show_hw=False):
    plt.figure(figsize=(12, 6))
    plt.plot(data["Sales"], label="Original Data", color="black",linestyle='--', linewidth=3)
    
    if show_ses:
        ses_values = ses(data["Sales"], alpha)
        plt.plot(ses_values, label="SES", color="blue",linewidth=3)
        
    if show_holt:
        holt_values = holt(data["Sales"], alpha, beta)
        plt.plot(holt_values, label="Holt", color="orange",linewidth=3)
        
    if show_hw:
        hw_values = holt_winters(data["Sales"], alpha, beta, gamma, season_length=seasonal_period)
        plt.plot(hw_values, label="Holt-Winters", color="green",linewidth=3)
        
    plt.title("Holt-Winters-Verfahren")
    plt.xlabel("Quarters")
    plt.ylabel("Sales")
    plt.style.use('bmh')
    plt.legend()
    plt.show()

# Interaktive Widgets
interact(
    plot_interactive,
    alpha=FloatSlider(value=0.3, min=0.01, max=1.0, step=0.01, description="Alpha (SES)"),
    beta=FloatSlider(value=0.3, min=0.01, max=1.0, step=0.01, description="Beta (Trend)"),
    gamma=FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description="Gamma (Seasonality)"),
    show_ses=Checkbox(value=False, description="Show SES"),
    show_holt=Checkbox(value=False, description="Show Holt"),
    show_hw=Checkbox(value=False, description="Show Holt-Winters")
)


interactive(children=(FloatSlider(value=0.3, description='Alpha (SES)', max=1.0, min=0.01, step=0.01), FloatSl…

<function __main__.plot_interactive(alpha=0.3, beta=0.3, gamma=0.1, show_ses=False, show_holt=False, show_hw=False)>

### Parameter Wahl Holt-Winters-Verfahren

- **Alpha**: steuert die Gewichtung des Niveaus --> wie schnell das Model auf Veränderung reagiert
    - hoher Alpha Wert berücksichtigt aktuelle Datenpunkte mehr  

- **Beta**: steuert die Gewichtung des Trends 
    - niedriger Wert wird der Trend langsamer aktualisiert --> reagiert nicht übermäßig auf Schwankungen

- **Gamma**: kontrolliert Gewichtung der saisonalen Komponente
    - niedriger gamma Wert hält die saison stabil und aktualisiert langsamer

Author:     Luis Wesinger  
Contact:    wesingerluis@gmail.com  edu: luis.wesinger@st.oth-regensburg.de  

This code ist published under the MIT-License.